# Hidden Factor Hidden Topics
## Amazon Kindle Store Reviews Dataset
Dataset Link: http://jmcauley.ucsd.edu/data/amazon/
##### Sample Review

In [2]:

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import json

In [4]:
DATASET_SIZE = 11586
DATASET_FILE = 'train.json'

dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]
data = []
for line in dataset_lines:
    data.append(json.loads(line)["reviewText"])
print data[0]

This has a very comfortable handle, does the job for which it is intended and is available at a reasonable price. I can't say that it makes weeding a pleasure but it does take some of the drudgery out of a necessary task. This is a worthwhile tool.


In [5]:
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                stop_words='english',
                                token_pattern=r'\b[a-zA-Z]{3,}\b',
                                max_df=0.5
                               )
dtm_tf = tf_vectorizer.fit_transform(data)

In [6]:
print (tf_vectorizer.get_feature_names()[:10])
print (dtm_tf.shape)

[u'aaa', u'aardvarks', u'aarp', u'aas', u'aback', u'abandon', u'abandoned', u'abandoning', u'abated', u'abbreviated']
(11586, 25257)


In [7]:
NUMBER_OF_TOPICS = 5


lda_tf = LatentDirichletAllocation(n_components=NUMBER_OF_TOPICS, random_state=18)
lda_tf.fit(dtm_tf)

/Users/bmunot/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=5, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=18,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [8]:
dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]

ReviewsByUser = {}
ItemReviews = {}
for line in dataset_lines:
    d = json.loads(line)
    user = d["reviewerID"]
    item = d["asin"]
    try: 
        ReviewsByUser[user] += ' ' + d["reviewText"]
    except:
        ReviewsByUser[user] = d["reviewText"]
    try: 
        ItemReviews[item] += ' ' + d["reviewText"]
    except:
        ItemReviews[item] = d["reviewText"]
        
print "done"

done


In [9]:
import numpy as np

WordToTopicMapping = {}

tf_feature_names = tf_vectorizer.get_feature_names()

rows = NUMBER_OF_TOPICS # topics count
columns = len(tf_feature_names) # Total number of words
    
M = np.zeros((rows, columns))
total_of_columns = np.sum(lda_tf.components_, axis=0)
print len(total_of_columns)
for i in range(columns):
    column_sum = total_of_columns[i]
    for j in range(rows):
        M[j][i] = lda_tf.components_[j][i]*1.0/column_sum

M = np.transpose(M)
DocVecRep = CountVectorizer(vocabulary=tf_feature_names) # document vector representation

25257


In [10]:
# print M

def MakeProbabilityDistribution(Gamma):
    column_sums = np.sum(Gamma, axis=0)
    return np.divide(Gamma, 1.0*column_sums)

Gamma_i = np.zeros((len(ItemReviews.keys()), NUMBER_OF_TOPICS)) #   #items   X  topics

j = 0
# Calculating latent factors for items
ItemToIndexMapping = {}
for item in ItemReviews.keys():
    ItemToIndexMapping[item] = j
    review = ItemReviews[item]
    review_vector = DocVecRep.fit_transform([review]).toarray()
    Gamma_i[j] = np.matmul(review_vector, M)
    j += 1


Gamma_i = np.transpose(Gamma_i)
Gamma_i = MakeProbabilityDistribution(Gamma_i)
print Gamma_i.shape

Gamma_u = np.zeros((len(ReviewsByUser.keys()), NUMBER_OF_TOPICS))
j = 0
# Calculating latent factors for users
UserToIndexMapping = {}
for user in ReviewsByUser.keys():
    UserToIndexMapping[user] = j
    review = ReviewsByUser[user]
    review_vector = DocVecRep.fit_transform([review]).toarray()
    Gamma_u[j] = np.matmul(review_vector, M)
    j += 1

Gamma_u = np.transpose(Gamma_u)
Gamma_u = MakeProbabilityDistribution(Gamma_u)
Gamma_u = np.transpose(Gamma_u)
print Gamma_u.shape

print UserToIndexMapping
print ItemToIndexMapping

    

(5, 962)
(1686, 5)
{u'A3DOPYDOS49I3T': 0, u'ADU56N9Z46FTF': 1, u'A6FIW5DREWU28': 2, u'ATMHSONJ55YN6': 3, u'A2068BC3ZXAVJQ': 4, u'A1CCBLW6NH70ID': 5, u'A1XLLEHKFY3TN2': 6, u'A14RYRR7ETV3UY': 7, u'A21I1N8JO0LQMO': 8, u'A1CXEPAONAMT7O': 9, u'A55LS2HWPQB0Q': 10, u'A1VK8XPVM7LRAO': 11, u'AIHQ4AXV5CT9N': 13, u'A2CC7WEZ0EZ07A': 14, u'A15A2KHW0KHJ8D': 15, u'A36MP37DITBU6F': 16, u'AYIU8KKZ6VOLU': 17, u'AFZJNYR931ZCN': 18, u'A1HYH206E18XVC': 19, u'A28YK7B27X3N0N': 20, u'A2XAWEMPKE9GDT': 21, u'A1YZ8KOTNU1BEU': 22, u'A1HTWL26WM2NVU': 23, u'AOMOTMG7R8AUO': 24, u'AG8DPYVTLT2C2': 25, u'A25LE7PMMSB629': 26, u'A1LSI6FHSA7VUS': 27, u'A1ZGHA3IJV01BK': 28, u'AX0ZTNX6KVJ40': 29, u'A17YPB3JICRUK2': 30, u'AQZ4PHIEEV5N5': 31, u'A3IUD6OUAS0A9K': 32, u'A1EH2S92FHWZRP': 33, u'A14DL8A4BT8YFK': 34, u'ADXVZUT1HHW2M': 35, u'A2PLSZ90SNG3FV': 36, u'AL3G7RTXMGLDQ': 37, u'A1WDM0EFA58H69': 1419, u'A1PNT1D0OBHLPV': 39, u'A2EF1YXI5VB622': 40, u'A3U8CM3MTMVHV1': 41, u'A277GP2U2TXH51': 42, u'AFVQ0AN26TVBV': 43, u'A1NFXS9QOKW

In [11]:

total_users = len(ReviewsByUser.keys())
total_items = len(ItemReviews.keys())

ActualRatingAvailability = np.zeros((total_users, total_items))

dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]

for line in dataset_lines:
    d = json.loads(line)
    userId = d["reviewerID"]
    itemId = d["asin"]
    user = UserToIndexMapping[userId]
    item = ItemToIndexMapping[itemId]

In [12]:
def PredictUserToItemRating(userId, itemId):
    userIndex = UserToIndexMapping[userId]
    itemIndex = ItemToIndexMapping[itemId]
    gamma_user = Gamma_u[userIndex]
    gamma_item = Gamma_i[:, itemIndex]
    return np.dot(gamma_user, gamma_item)

def IsActuallyRated(userId, itemId):
    user = UserToIndexMapping[userId]
    item = ItemToIndexMapping[itemId]
    
    return ActualRatingAvailability[user][item] == 1


def RecommendItemsToUser(userId):
    itemsList = ItemToIndexMapping.keys()
    predictions = []
    for item in itemsList:
        if not IsActuallyRated(userId, item):
            predictions.append((item, PredictUserToItemRating(userId, item)))
    sorted_by_predictions = sorted(predictions, key=lambda tup: tup[1], reverse=True)
    print len(sorted_by_predictions)
    print sorted_by_predictions[:min(10, len(predictions))]
    return sorted_by_predictions[:min(10, len(predictions))]

def GetRecommendationsForUser(userId):
    predictions = RecommendItemsToUser(userId)
    return [_[0] for _ in predictions]

# print GetRecommendationsForUser('A2QK1U70OJ74P')

In [13]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/bmunot/anaconda2/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      32.143271        1       1  0.055371  0.102054
0      26.949679        1       2  0.051690  0.053288
2      22.453023        1       3  0.070021  0.115151
3      13.110279        1       4  0.117981 -0.233101
1       5.343748        1       5 -0.295062 -0.037392, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
7918   Default  4299.000000       feeder  4299.000000  30.0000  30.0000
22903  Default  3888.000000         trap  3888.000000  29.0000  29.0000
16695  Default  3988.000000      product  3988.000000  28.0000  28.0000
13871  Default  1269.000000        mower  1269.000000  27.0000  27.0000
24405  Default  3470.000000        water  3470.000000  26.0000  26.0000
2031   Default  2201.000000        birds  2201.000000  25.0000  25.0000
5432   Default  1293.000000         deer  1293.000000  24.0000  24.0000
15971  Default  2141.000000       plants  2141.000000  23.0000  23.0000
3750   Default  1454.000000        clean  1454.000000  22.0000  22.0000
13831  Default  1861.000000        mouse  1861.000000  21.0000  21.0000
22924  Default  1817.000000        traps  1817.000000  20.0000  20.0000
20623  Default  1370.000000        spray  1370.000000  19.0000  19.0000
1705   Default  1732.000000      battery  1732.000000  18.0000  18.0000
12218  Default  1128.000000         lawn  1128.000000  17.0000  17.0000
7932   Default  1254.000000      feeders  1254.000000  16.0000  16.0000
8965   Default  2258.000000       garden  2258.000000  15.0000  15.0000
13410  Default  1574.000000         mice  1574.000000  14.0000  14.0000
10470  Default  2641.000000         hose  2641.000000  13.0000  13.0000
2368   Default   850.000000       bottle   850.000000  12.0000  12.0000
1510   Default  1371.000000         bait  1371.000000  11.0000  11.0000
10598  Default   605.000000  hummingbird   605.000000  10.0000  10.0000
9459   Default  1062.000000        grass  1062.000000   9.0000   9.0000
23035  Default  1156.000000      trimmer  1156.000000   8.0000   8.0000
9545   Default  2155.000000        grill  2155.000000   7.0000   7.0000
8998   Default  1118.000000          gas  1118.000000   6.0000   6.0000
2013   Default  1254.000000         bird  1254.000000   5.0000   5.0000
9196   Default   560.000000        glass   560.000000   4.0000   4.0000
17566  Default   628.000000          red   628.000000   3.0000   3.0000
16364  Default  1088.000000        power  1088.000000   2.0000   2.0000
4799   Default  1299.000000        cover  1299.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
7224    Topic5    43.296455      enzymes    44.077881   2.9114  -6.8962
4018    Topic5    34.628298     coloring    35.408903   2.9070  -7.1196
13877   Topic5   156.938982       mowers   160.616227   2.9061  -5.6084
14908   Topic5    95.366678       ounces    97.928721   2.9027  -6.1065
4631    Topic5   192.727465       copper   207.529700   2.8552  -5.4029
15904   Topic5   278.918052          pit   307.070029   2.8331  -5.0333
13871   Topic5  1037.315611        mower  1269.288499   2.7274  -3.7198
17566   Topic5   463.482926          red   628.630576   2.6245  -4.5255
3223    Topic5    89.057098      catcher   101.211978   2.8013  -6.1749
2368    Topic5   503.550128       bottle   850.543731   2.4051  -4.4425
15621   Topic5   255.459695        perky   474.274141   2.3105  -5.1212
1634    Topic5   292.146203         base   578.990021   2.2452  -4.9870
12218   Topic5   468.624576         lawn  1128.035480   2.0508  -4.5144
3750    Topic5   543.282505        clean  1454.259013   1.9446  -4.3666
11672   Topic5   107.039913          jar   142.851491   2.6406  -5.9910
14903   Topic5   125.769879        ounce   182.281064   2.5581  -5.8298
1769    Topic5   152.529163    beautiful   252.2242

In [103]:

# Converting json to votes file format

dataset_file = open(DATASET_FILE)
dataset_lines = dataset_file.readlines()[:DATASET_SIZE]

output_file = open('Kindle.votes', 'w')
for line in dataset_lines:
    d = json.loads(line)
    output_file.write(d["reviewerID"] + ' ' + d["asin"]+ ' ' + str(d["overall"]) + ' ' + str(d["unixReviewTime"]) + ' ' + str(len(d["reviewText"].split())) + ' ' + d["reviewText"] + '\n')
output_file.close()